# Zero to Singularity: Create, Tune, Deploy and Scale a Deep Neural Network in 90 Minutes

This notebook is part of a masterclass held at IBM Think on 13th of February 2019 in San Fransisco
In this exercise you will train a Keras DeepLearning model running on top of TensorFlow. 

Note: For sake of bringing the training runtime down we've done two things

1) Used a softmax regression model over a Convolutional Neural Network 

2) Trained only for one epoch instead of 20

This leads to approx. 5% less accuracy


Authors

Romeo Kienzler - Chief Data Scientist, IBM Watson IoT

Krishnamurthy Arthanarisamy - Architect, Watson Machine Learning Software Lab, Bangalore


# Prerequisites

Please make sure the currently installed version of Keras and Tensorflow are matching the requirememts, if not, please run the two PIP commands below in order to re-install. Please restart the kernal before proceeding, please re-check if the versions are matching.

In [4]:
import keras
print('Current:\t', keras.__version__)
print('Expected:\t 2.2.5 ')

Using TensorFlow backend.


Current:	 2.2.5
Expected:	 2.2.5 


In [5]:
import tensorflow as tf
print('Current:\t', tf.__version__)
print('Expected:\t 1.15.0')

Current:	 1.15.0
Expected:	 1.15.0


# IMPORTANT !!!

If you ran the two lines below please restart your kernel (Kernel->Restart & Clear Output)

In [2]:
!pip install keras==2.2.5 

     |████████████████████████████████| 336 kB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 1.4 MB/s eta 0:00:01


In [3]:
!pip install tensorflow==1.15.0

     |████████████████████████████████| 412.3 MB 35 kB/s  eta 0:00:01   |▋                               | 8.5 MB 1.9 MB/s eta 0:03:28     |█▍                              | 17.9 MB 1.9 MB/s eta 0:03:25     |█▊                              | 21.8 MB 1.8 MB/s eta 0:03:36     |██                              | 26.6 MB 2.2 MB/s eta 0:02:53     |██▌                             | 32.5 MB 2.3 MB/s eta 0:02:48     |████▌                           | 58.4 MB 2.4 MB/s eta 0:02:28     |███████                         | 88.8 MB 2.0 MB/s eta 0:02:43     |████████                        | 104.0 MB 2.0 MB/s eta 0:02:34     |███████████▏                    | 143.4 MB 2.0 MB/s eta 0:02:18     |█████████████                   | 166.5 MB 1.8 MB/s eta 0:02:15     |███████████████▌                | 200.2 MB 2.2 MB/s eta 0:01:37     |████████████████▌               | 213.1 MB 2.0 MB/s eta 0:01:42     |█████████████████               | 219.6 MB 2.3 MB/s eta 0:01:23     |███████████████████▊            | 253.

# 1.0 Train a MNIST digits recognition model
We start with some global parameters and imports

In [9]:
#some learners constantly reported 502 errors in Watson Studio. 
#This is due to the limited resources in the free tier and the heavy resource consumption of Keras.
#This is a workaround to limit resource consumption

from keras import backend as K

K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))

AttributeError: module 'keras.backend' has no attribute 'tf'

In [10]:
import keras
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.optimizers import RMSprop
from keras.layers import LeakyReLU

from keras import backend as K
import numpy as np

In [11]:
batch_size = 128
num_classes = 10
epochs = 1

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


11493376/11490434 [==============================] - 10s 1us/step


# Training a simple model
First we'll train a simple softmax regressor and check what accuracy we get

In [12]:
model = Sequential()
model.add(Dense(512, input_shape=(784,)))
model.add(Dense(num_classes, activation='softmax'))



model.compile(loss='categorical_crossentropy',
        optimizer='rmsprop',
        metrics=['accuracy'])

model.fit(x_train, y_train,
        batch_size=batch_size,
        epochs=epochs,
        verbose=1,
        validation_data=(x_test, y_test))
        
score = model.evaluate(x_test, y_test, verbose=0)

print('\n')
print('Accuracy:',score[1])






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 60000 samples, validate on 10000 samples
Epoch 1/1





60000/60000 [==============================] - 6s 108us/step - loss: 0.3829 - acc: 0.8890 - val_loss: 0.3050 - val_acc: 0.9145


Accuracy: 0.9145


In [13]:
#some cleanup from the previous run
!rm -f ker_*
!rm -f my_best_model.tgz

You should see an accuracy of approximately 90%. Now lets define a hyper-parameter grid including different activation functions and gradient descent optimizers. We’re optimizing over the grid using grid search (nested for loops) and store each model variant in a file. We then decide for the best one in order to deploy to IBM Watson Machine Learning.

In [14]:
#define parameter grid

activation_functions_layer_1 = ['sigmoid','tanh','relu']
opimizers = ['rmsprop','adagrad','adadelta']

#optimize over parameter grid (grid search)

for activation_function_layer_1 in activation_functions_layer_1:
    for opimizer in opimizers:
        
        model = Sequential()
        model.add(Dense(512, activation = activation_function_layer_1, input_shape=(784,)))
        model.add(Dense(num_classes, activation='softmax'))



        model.compile(loss='categorical_crossentropy',
              optimizer=opimizer,
              metrics=['accuracy'])

        model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
        
        score = model.evaluate(x_test, y_test, verbose=0)
        save_path = "ker_func_mnist_model_2.%s.%s.%s.h5" % (activation_function_layer_1,opimizer,score[1])
        model.save(save_path)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 5s 87us/step - loss: 0.4210 - acc: 0.8869 - val_loss: 0.2613 - val_acc: 0.9241
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 5s 90us/step - loss: 0.4129 - acc: 0.8888 - val_loss: 0.2849 - val_acc: 0.9193
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 7s 111us/step - loss: 0.5391 - acc: 0.8579 - val_loss: 0.3273 - val_acc: 0.9061
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 5s 85us/step - loss: 0.3377 - acc: 0.9006 - val_loss: 0.2560 - val_acc: 0.9197
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 6s 103us/step - loss: 0.3265 - acc: 0.9076 - val_loss: 0.2273 - val_acc: 0.9355
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60

# Model evaluation
Let's have a look at all the models and see which hyper parameter configuration was the best one. You should see that relu and rmsprop gives you > 95% of accuracy on the validation set

In [15]:
ls -ltr ker_*

-rw-r--r-- 1 daniel daniel 3276560 Jul 24 10:40 ker_func_mnist_model_2.sigmoid.rmsprop.0.9241.h5
-rw-r--r-- 1 daniel daniel 3276560 Jul 24 10:40 ker_func_mnist_model_2.sigmoid.adagrad.0.9193.h5
-rw-r--r-- 1 daniel daniel 4905120 Jul 24 10:40 ker_func_mnist_model_2.sigmoid.adadelta.0.9061.h5
-rw-r--r-- 1 daniel daniel 3276560 Jul 24 10:40 ker_func_mnist_model_2.tanh.rmsprop.0.9197.h5
-rw-r--r-- 1 daniel daniel 3276568 Jul 24 10:41 ker_func_mnist_model_2.tanh.adagrad.0.9355.h5
-rw-r--r-- 1 daniel daniel 4905128 Jul 24 10:41 ker_func_mnist_model_2.tanh.adadelta.0.9214.h5
-rw-r--r-- 1 daniel daniel 3276568 Jul 24 10:41 ker_func_mnist_model_2.relu.rmsprop.0.9616.h5
-rw-r--r-- 1 daniel daniel 3276568 Jul 24 10:41 ker_func_mnist_model_2.relu.adagrad.0.9567.h5
-rw-r--r-- 1 daniel daniel 4905128 Jul 24 10:41 ker_func_mnist_model_2.relu.adadelta.0.9558.h5


Now it's time to create a tarball out of your favorite model, please replace the name of your favorite model H5 file with “please-put-me-here”

In [16]:
!tar -zcvf my_best_model.tgz please-put-me-here.h5

tar: please-put-me-here.h5: Cannot stat: No such file or directory
tar: Exiting with failure status due to previous errors


## 2.0 Save the trained model to WML Repository

We will use `watson_machine_learning_client` python library to save the trained model to WML Repository, to deploy the saved model and to make predictions using the deployed model.</br>


`watson_machine_learning_client` can be installed using the following `pip` command in case you are running outside Watson Studio:

`!pip install watson-machine-learning-client --upgrade`

In [18]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

ContextualVersionConflict: (docutils 0.16 (/home/daniel/anaconda3/lib/python3.7/site-packages), Requirement.parse('docutils<0.16,>=0.10'), {'ibm-cos-sdk-core'})

Please go to https://cloud.ibm.com/,  login, click on the “Create Resource” button. From the “AI” category, please choose “Machine Learning”. Wait for the “Create” button to activate and click on “Create”. Click on “Service Credentials”, then “New Credential”, then “Add”. From the new entry in the table, under “ACTIONS”, please click on “View Credentials”. Please copy the whole JSON object to your clipboard. Now just paste the JSON object below so that you are able to use your personal instance of Watson Machine Learning.

In [21]:
wml_credentials=####

SyntaxError: invalid syntax (<ipython-input-21-623c9daa5ae8>, line 1)

In [20]:
client = WatsonMachineLearningAPIClient(wml_credentials)

NameError: name 'WatsonMachineLearningAPIClient' is not defined

In [ ]:
model_props = {client.repository.ModelMetaNames.AUTHOR_NAME: "IBM", 
               client.repository.ModelMetaNames.AUTHOR_EMAIL: "ibm@ibm.com", 
               client.repository.ModelMetaNames.NAME: "KK3_clt_keras_mnist",
               client.repository.ModelMetaNames.FRAMEWORK_NAME: "tensorflow",
               client.repository.ModelMetaNames.FRAMEWORK_VERSION: "1.15" ,
               client.repository.ModelMetaNames.FRAMEWORK_LIBRARIES: [{"name": "keras", "version": "2.2.5"}]
              }

In [ ]:
published_model = client.repository.store_model(model="my_best_model.tgz", meta_props=model_props)


In [ ]:
published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)

## 3.0 Deploy the Keras model

In [ ]:
client.deployments.list()

To keep your environment clean, just delete all deployments from previous runs

In [ ]:
client.deployments.delete("PASTE_YOUR_GUID_HERE_IF_APPLICABLE")

In [ ]:
created_deployment = client.deployments.create(published_model_uid, name="k1_keras_mnist_clt1")

## Test the model

In [ ]:
#scoring_endpoint = client.deployments.get_scoring_url(created_deployment)
scoring_endpoint = created_deployment['entity']['scoring_url']
print(scoring_endpoint)

In [ ]:
x_score_1 = x_test[23].tolist()
print('The answer should be: ',np.argmax(y_test[23]))
scoring_payload = {'values': [x_score_1]}

In [ ]:
predictions = client.deployments.score(scoring_endpoint, scoring_payload)
print('And the answer is!... ',predictions['values'][0][1])